<a href="https://colab.research.google.com/github/lemmingy/My_pipelines/blob/main/liner_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install category_encoders

In [2]:
%cd /content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model

/content/drive/My Drive/Colab Notebooks/Kaggle/Library/Input_data/liner_model


In [3]:
import pandas as pd
import numpy as np
# pd.set_option('display.max_columns', 500)#表示列を増加。

df_train = pd.read_csv("train.csv")
# df_train= pd.read_csv('XXX.csv',parse_dates=['issue_d','earliest_cr_line'])#日付がある場合にはparseする。
df_train.head()

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Respondent,ConvertedSalary,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,3,70841.0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,...,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,7,21426.0,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,...,6.0,1.0,7.0,5.0,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,8,41671.0,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",

In [4]:
# Testのデータの傾向をよくみる。Trainingデータと比べて違う所はないか？トレーニングデータをそのまま使って良いのか？正しいパーティションは何か？
df_test = pd.read_csv("test.csv")
df_test.head()

,Respondent,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,81,No,Yes,Germany,No,Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",100 to 499 employees,Back-end developer;Desktop or enterprise appli...,9-11 years,9-11 years,Moderately satisfied,Extremely satisfied,Working as a product manager or project manager,"I’m not actively looking, but I am open to new...",Less than a year ago,9.0,5.0,10.0,2.0,3.0,4.0,6.0,1.0,8.0,7.0,1.0,10.0,11.0,6.0,8.0,9.0,2.0,5.0,4.0,3.0,7.0,5.0,1.0,...,3.0,2.0,6.0,5.0,NaN,NaN,Nobody,"I don't care about it, or I haven't thought ab...",Depends on what it is,"Yes, but only within the company",Upper management at the company/organization,Unsure / I don't know,4,A few times per week,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,A little bit interested,Somewhat interested,Not at all interested,Not at all interested,Not at all interested,Between 6:01 - 7:00 AM,5 - 8 hours,1 - 2 hours,Never,NaN,I don't typically exercise,Male,Straight or heterosexual,"Secondary school (e.g. American high school, G...",White or of European descent,35 - 44 years old,Yes,NaN,The survey was too long,Neither easy nor difficult
1,85,Yes,Yes,France,"Yes, part-time",Employed part-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10 to 19 employees,Back-end developer;Front-end developer;Student,9-11 years,0-2 years,Neither satisfied nor dissatisfied,Moderately dissatisfied,Working as a founder or co-founder of my own c...,"I’m not actively looking, but I am open to new...",Less than a year ago,1.0,10.0,4.0,5.0,2.0,6.0,3.0,7.0,8.0,9.0,1.0,3.0,9.0,11.0,10.0,8.0,4.0,7.0,6.0,2.0,5.0,5.0,1.0,...,5.0,4.0,7.0,2.0,"Evolving definitions of ""fairness"" in algorith...",Artificial intelligence surpassing human intel...,Nobody,I'm excited about the possibilities more than ...,No,Depends on what it is,NaN,Yes,10 (Very Likely),Multiple times per day,No,NaN,Yes,NaN,5,I'm not sure,Extremely interested,Extremely interested,A little bit interested,Very interested,Very interested,Between 5:00 - 6:00 AM,Over 12 hours,1 - 2 hours,Daily or almost every day,NaN,I don't typically exercise,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,18 - 24 years old,No,NaN,The survey was too long,Very easy
2,92,Yes,Yes,Germany,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",500 to 999 employees,Full-stack developer,6-8 years,0-2 years,Slightly dissatisfied,Slightly satisfied,Doing the same work,I am actively looking for a job,Between 1 and 2 years ago,6.0,2.0,8.0,1.0,3.0,5.0,10.0,4.0,9.0,7.0,1.0,9.0,2.0,7.0,10.0,3.0,11.0,4.0,8.0,5.

In [5]:
# ターゲットを分けておく。IDも分けておく
target = "ConvertedSalary"
id = "Respondent"
try:
  df_train_y = df_train[target]
  df_train.drop(columns = target, inplace = True)
  df_train_id = df_train[id]
  df_test_id = df_test[id]
  df_train.drop(columns = id, inplace = True)
  df_test.drop(columns = id, inplace = True)  
except:
  pass

print(df_train.shape, df_test.shape, df_train_y.shape)

(42893, 112) (4809, 112) (42893,)


In [6]:
# 時間特徴量の簡易処理（あれば。事前に時間特徴量にparse_datesで変換しておく。）
# 時間を分解
def sep_time(df, col, col2=0):
  df[col+"_y"]=df[col].dt.year
  df[col+"_y"]=df[col].dt.month
  df[col+"_y"]=df[col].dt.day
  if col2!=0:
    df[col2+"_y"]=df[col2].dt.year
    df[col2+"_y"]=df[col2].dt.month
    df[col2+"_y"]=df[col2].dt.day
    df[col+"-"+col2]=df[col] - df[col2]
  return df

def time_encode(df, col):
    # この方法だと場合によって最大値が変化するデータでは正確な値は出ない
    # 例：月の日数が30日や31日の場合がある
    df[col + '_cos'] = np.cos(2 * np.pi * df[col] / df[col].max())
    df[col + '_sin'] = np.sin(2 * np.pi * df[col] / df[col].max())
    return df


In [7]:
# 数値カラム、カテゴリカラム、テキストコラムのリストを取得
def get_columns(df):
  lis_cols_num = [ col for col in df.columns if df[col].dtype == "float64" or df[col].dtype == "int64" ]
  lis_cols_cat = [ col for col in df.columns if df[col].dtype == "object" ]
  return lis_cols_num, lis_cols_cat

lis_cols_num, lis_cols_cat  =  get_columns(df_train)

In [8]:
# 数値カラムの外れ値の影響を減らすためのクリッピング
def clip(df, lis_cols_num):
  for col in lis_cols_num:
    upperbound, lowerbound= np.percentile(df[col],[1,99])
    df[col]=np.clip(df[col],upperbound,lowerbound)
    return df

df_train = clip(df_train, lis_cols_num)
df_test = clip(df_test, lis_cols_num)
df_train.head()

/usr/local/lib/python3.6/dist-packages/pandas/core/generic.py:7212: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  result[mask] = np.nan


,Hobby,OpenSource,Country,Student,Employment,FormalEducation,UndergradMajor,CompanySize,DevType,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,...,AdsPriorities4,AdsPriorities5,AdsPriorities6,AdsPriorities7,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy
0,Yes,Yes,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...","10,000 or more employees",Database administrator;DevOps specialist;Full-...,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,...,4.0,6.0,7.0,2.0,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
1,Yes,No,South Africa,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...","10,000 or more employees",Data or business analyst;Desktop or enterprise...,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,...,6.0,1.0,7.0,5.0,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy
2,Yes,No,United Kingdom,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof..

In [9]:
#カテゴリ列について、trainingにしか現れないカテゴリ、もしくはtestにしか現れないカテゴリを可視化
def check_cat(df_train, df_test, col):
  train_s = set(df_train[col].unique())
  test_s = set(df_test[col].unique())
  only_training = train_s - test_s
  only_test = test_s - train_s
  print("trainingのみ:"+str(only_training))
  print("testのみ:"+str(only_test))
  return only_training, only_test


only_training, only_test = check_cat(df_train, df_test, "Country")

trainingのみ:{'Algeria', 'Venezuela, Bolivarian Republic of...', 'Saudi Arabia', 'Taiwan', 'Kuwait', 'Sri Lanka', 'Azerbaijan', 'Fiji', 'Togo', 'Nigeria', 'Japan', 'Libyan Arab Jamahiriya', 'Belarus', 'Rwanda', 'Canada', 'South Africa', 'Egypt', 'Tajikistan', 'Brazil', 'Israel', 'Yemen', 'Austria', 'Russian Federation', 'Afghanistan', 'Bangladesh', 'United Republic of Tanzania', 'Dominica', 'United States', 'Georgia', 'Nicaragua', 'Cambodia', 'Viet Nam', 'Swaziland', 'Norway', 'Spain', 'Thailand', 'Hong Kong (S.A.R.)', 'Nepal', 'Trinidad and Tobago', 'Argentina', 'Denmark', 'Romania', 'Monaco', 'South Korea', 'Sweden', 'Australia', 'Saint Lucia', 'Marshall Islands', 'United Kingdom', 'Sudan', 'Tunisia', 'United Arab Emirates', 'Luxembourg', 'Italy', 'Guatemala', 'Kazakhstan', 'Uzbekistan', 'Colombia', 'India', 'Oman', 'Other Country (Not Listed Above)', 'Singapore', 'Ethiopia', 'Mozambique', 'Montenegro', 'Syrian Arab Republic', 'Portugal', 'Mongolia', 'Kenya', 'Myanmar', 'Uganda', 'Bhut

In [10]:
#カテゴリ特徴量について、一定数以上いない or Trainingにしか出てこない物をまとめてOthersに落とす。
def category_check(df_train, df_test, t_col):
  print("Training_data :", df_train[t_col].unique())
  print("Test_data :", df_test[t_col].unique())
  print("Only Training data :", set(df_train[t_col].unique()) - set(df_test[t_col].unique()))
  print("Only Test data :", set(df_test[t_col].unique()) - set(df_train[t_col].unique()))
  return 



In [11]:
#次のセルの準備。本来数字のカテゴリ特徴量を数字に変換する準備。個別に確認して検討していく。
t_col = "CompanySize"
category_check(df_train, df_test, t_col)

Training_data : ['10,000 or more employees' '10 to 19 employees' '100 to 499 employees'
 '20 to 99 employees' '1,000 to 4,999 employees' '500 to 999 employees'
 '5,000 to 9,999 employees' 'Fewer than 10 employees' nan]
Test_data : ['100 to 499 employees' '10 to 19 employees' '500 to 999 employees'
 '10,000 or more employees' '20 to 99 employees' 'Fewer than 10 employees'
 '5,000 to 9,999 employees' '1,000 to 4,999 employees' nan]
Only Training data : set()
Only Test data : set()


In [12]:
#本来数字であるカテゴリ特徴量を数字に変換する。lis_cols_catから抜き、lis_cols_numにいれる
def order_num(df, col, lis):
  now_lis = df[col].unique()
  dic = {}
  for a, b in zip(now_lis, lis):
    dic[a] = b
  df[col].replace(dic, inplace = True)
  return df

t_col = "CompanySize"
order_lis=[10000, 10, 100, 20, 1000, 500, 5000, 0, np.nan]
df_train = order_num(df_train, t_col, order_lis)
df_test = order_num(df_test, t_col, order_lis)
lis_cols_cat.remove(t_col)
lis_cols_num.append(t_col)
df_train[t_col].unique()

array([10000.,    10.,   100.,    20.,  1000.,   500.,  5000.,     0.,
          nan])

In [13]:
# カテゴリリストの中からテキストか判別する。trainingまたはtestにthreshold以上のカーディナリティをテキストにする。でもtrainingとtestで数が違うならむしろデータをチェックした方が良い。
threshold = 100

def find_text(df, df2, col_lis, thre):
  txt_lis = []
  for col in col_lis:
    n1 = df[col].nunique()
    n2 = df2[col].nunique()
    print(col, "// training:", n1, "test:", n2)
    if n1 >= thre or n2> thre:
      txt_lis.append(col)
      col_lis.remove(col)
  return col_lis, txt_lis
      

lis_cols_cat, lis_cols_txt = find_text(df_train, df_test, lis_cols_cat, threshold)
print(lis_cols_cat, lis_cols_txt)

Hobby // training: 2 test: 2
OpenSource // training: 2 test: 2
Country // training: 155 test: 2
Employment // training: 6 test: 6
FormalEducation // training: 9 test: 9
UndergradMajor // training: 12 test: 12
DevType // training: 5695 test: 1368
YearsCodingProf // training: 11 test: 11
JobSatisfaction // training: 7 test: 7
CareerSatisfaction // training: 7 test: 7
HopeFiveYears // training: 7 test: 7
JobSearchStatus // training: 3 test: 3
LastNewJob // training: 5 test: 5
UpdateCV // training: 8 test: 8
Currency // training: 19 test: 10
SalaryType // training: 3 test: 3
CurrencySymbol // training: 111 test: 11
TimeFullyProductive // training: 6 test: 6
TimeAfterBootcamp // training: 8 test: 8
AgreeDisagree1 // training: 5 test: 5
AgreeDisagree2 // training: 5 test: 5
AgreeDisagree3 // training: 5 test: 5
FrameworkWorkedWith // training: 811 test: 287
NumberMonitors // training: 5 test: 5
CheckInCode // training: 6 test: 6
AdBlocker // training: 3 test: 3
AdBlockerDisable // training: 

In [14]:
lis_cols_txt

['Country', 'DevType', 'CurrencySymbol', 'FrameworkWorkedWith']

In [15]:
# テキストの特徴量に対してElastic-netでstacked predictionでメタ特徴量を作成する。
# 分類にするならLogisticRegressionのペナルティを編集してElastic-netに。https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LogisticRegression.html#sklearn.linear_model.LogisticRegression
# Foldを変えるならはここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.model_selection
# 線形モデルを変えるならここを参照 https://scikit-learn.org/stable/modules/classes.html#module-sklearn.linear_model
# メトリクスを変えるならここを参照https://scikit-learn.org/stable/modules/classes.html#module-sklearn.metrics

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import StratifiedKFold
from scipy.sparse import hstack
from sklearn.linear_model import ElasticNet
from sklearn.metrics import mean_squared_error
# from sklearn.metrics import roc_auc_score

SEED = 71
NFOLDS = 5
fold = StratifiedKFold(n_splits=NFOLDS, random_state=SEED, shuffle=True)
fold = StratifiedKFold(n_splits=NFOLDS, random_state=SEED, shuffle=True)

def text_convert(df_train, df_train_y, df_test, col, fold):
  txt_train = df_train[col]
  txt_train.fillna("nan", inplace = True)
  txt_test = df_test[col]
  txt_test.fillna("nan", inplace = True)
  
  tfidf = TfidfVectorizer(max_features=10000)
  txt_train = tfidf.fit_transform(txt_train)
  txt_test = tfidf.transform(txt_test)

  oof_train = np.zeros(len(df_train))
  oof_test = np.zeros(len(df_test))

  for i, (train_idx, test_idx) in enumerate(fold.split(df_train, df_train_y)):
      train_x, train_y = txt_train[train_idx], df_train_y.iloc[train_idx]
      test_x, test_y = txt_train[test_idx], df_train_y.iloc[test_idx]
      
      # # LogisticRegression for 分類問題
      # clf = LogisticRegression(penalty='elasticnet',solver='sag')
      # clf.fit(X_tr, y_tr)
      # y_pred = clf.predict_proba(test_x)[:,1]
      # score = roc_auc_score(test_y, y_pred)
      # print('CV Score of Fold_%d is %f' % (i, score))
      # oof_test += clf.predict_proba(txt_test)[:,1]

      # ElasticNet for 回帰問題
      clf = ElasticNet(random_state=0)
      clf.fit(train_x, train_y)
      y_pred = clf.predict(test_x)
      # score = mean_squared_error(test_y, y_pred)
      # print('CV Score of Fold_%d is %f' % (i, score))
      oof_test += clf.predict(txt_test)

      oof_train[test_idx] = y_pred
  oof_test /= NFOLDS

  df_train[col+"_pred"] = oof_train
  df_test[col+"_pred"] = oof_test
  df_train.drop(columns = col, inplace=True)
  df_test.drop(columns = col, inplace=True)
  return df_train, df_test

for col in lis_cols_txt:
  df_train, df_test = text_convert(df_train, df_train_y, df_test, col, fold)
df_train.head()

/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


,Hobby,OpenSource,Student,Employment,FormalEducation,UndergradMajor,CompanySize,YearsCoding,YearsCodingProf,JobSatisfaction,CareerSatisfaction,HopeFiveYears,JobSearchStatus,LastNewJob,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,...,AIDangerous,AIInteresting,AIResponsible,AIFuture,EthicsChoice,EthicsReport,EthicsResponsible,EthicalImplications,StackOverflowRecommend,StackOverflowVisit,StackOverflowHasAccount,StackOverflowParticipate,StackOverflowJobs,StackOverflowDevStory,StackOverflowJobsRecommend,StackOverflowConsiderMember,HypotheticalTools1,HypotheticalTools2,HypotheticalTools3,HypotheticalTools4,HypotheticalTools5,WakeTime,HoursComputer,HoursOutside,SkipMeals,ErgonomicDevices,Exercise,Gender,SexualOrientation,EducationParents,RaceEthnicity,Age,Dependents,MilitaryUS,SurveyTooLong,SurveyEasy,Country_pred,DevType_pred,CurrencySymbol_pred,FrameworkWorkedWith_pred
0,Yes,Yes,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","A natural science (ex. biology, chemistry, phy...",10000.0,30 or more years,18-20 years,Moderately dissatisfied,Neither satisfied nor dissatisfied,Working in a different or more specialized tec...,I am actively looking for a job,More than 4 years ago,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,...,Increasing automation of jobs,Increasing automation of jobs,The developers or the people creating the AI,I'm excited about the possibilities more than ...,Depends on what it is,Depends on what it is,Upper management at the company/organization,Yes,10 (Very Likely),A few times per month or weekly,Yes,A few times per month or weekly,Yes,"No, I have one but it's out of date",7,Yes,A little bit interested,A little bit interested,A little bit interested,A little bit interested,A little bit interested,Between 6:01 - 7:00 AM,5 - 8 hours,30 - 59 minutes,Never,Ergonomic keyboard or mouse,Daily or almost every day,Male,Straight or heterosexual,"Bachelor’s degree (BA, BS, B.Eng., etc.)",White or of European descent,35 - 44 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,101258.352106,97988.065984,95651.377054,95904.072252
1,Yes,No,"Yes, part-time",Employed full-time,Some college/university study without earning ...,"Computer science, computer engineering, or sof...",10000.0,6-8 years,0-2 years,Slightly satisfied,Moderately satisfied,Working in a different or more specialized tec...,"I’m not actively looking, but I am open to new...",Between 1 and 2 years ago,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,...,Algorithms making important decisions,Algorithms making important decisions,The developers or the people creating the AI,I'm excited about the possibilities more than ...,No,"Yes, but only within the company",Upper management at the company/organization,Yes,10 (Very Likely),Daily or almost daily,Yes,Less than once per month or monthly,"No, I knew that Stack Overflow had a jobs boar...","No, I know what it is but I don't have one",NaN,Yes,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Extremely interested,Before 5:00 AM,Over 12 hours,1 - 2 hours,Never,NaN,3 - 4 times per week,Male,Straight or heterosexual,Some college/university study without earning ...,White or of European descent,18 - 24 years old,Yes,NaN,The survey was an appropriate length,Somewhat easy,88312.214172,95001.274095,89516.616580,93328.866628
2,Yes,No,No,Employed full-time,"Bachelor’s degree (BA, BS, B.Eng., etc.)","Computer science, computer engineering, or sof...",10.0,6-8 years,3-5 years,Moderately satisfied,Slig

In [16]:
# カテゴリ特徴量の変換　https://www.analyticsvidhya.com/blog/2020/08/types-of-categorical-data-encoding/
import category_encoders as ce
import warnings
warnings.simplefilter('ignore')

def cat_encode_for_liner(df, df2, col):
  encode = ce.OneHotEncoder(cols = col, handle_unknown = 'return_nan', return_df = True, use_cat_names = True)
  df_tmp = encode.fit_transform(df[col])
  df = pd.concat([df, df_tmp], axis = 1)
  df.drop(columns = col, inplace = True)
  df_tmp = encode.transform(df2[col])
  df2 = pd.concat([df2, df_tmp], axis = 1)
  df2.drop(columns = col, inplace = True)
  return df, df2

for col in lis_cols_cat:
  df_train, df_test = cat_encode_for_liner(df_train, df_test, col)
print(df_train.shape, df_test.shape)
df_train.head()

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


(42893, 1562) (4809, 1562)


,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,...,RaceEthnicity_Black or of African descent;East Asian;Hispanic or Latino/Latina,RaceEthnicity_East Asian;Middle Eastern;South Asian,"RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Native American, Pacific Islander, or Indigenous Australian","RaceEthnicity_East Asian;Native American, Pacific Islander, or Indigenous Australian;South Asian","RaceEthnicity_Black or of African descent;East Asian;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",RaceEthnicity_Black or of African descent;East Asian;Hispanic or Latino/Latina;White or of European descent,"RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Native American, Pacific Islander, or Indigenous Australian;White or of European descent",RaceEthnicity_Hispanic or Latino/Latina;South Asian;White or of European descent,RaceEthnicity_Black or of African descent;East Asian;South Asian,"RaceEthnicity_Black or of African descent;East Asian;Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;White or of European descent","RaceEthnicity_Black or of African descent;East Asian;Native American, Pacific Islander, or Indigenous Australian","RaceEthnicity_Native American, Pacific Islander, or Indigenous Australian;South Asian","RaceEthnicity_Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;White or of European descent","RaceEthnicity_Black or of African descent;East Asian;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Middle Eastern,"RaceEthnicity_East Asian;Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",Age_35 - 44 years old,Age_18 - 24 years old,Age_25 - 34 years old,Age_45 - 54 years old,Age_nan,Age_55 - 64 years old,Age_Under 18 years old,Age_65 years or older,Dependents_Yes,Dependents_No,Dependents_nan,MilitaryUS_nan,MilitaryUS_No,MilitaryUS_Yes,SurveyTooLong_The survey was an appropriate length,SurveyTooLong_The survey was too long,SurveyTooLong_nan,SurveyTooLong_The survey was too short,SurveyEasy_Somewhat easy,SurveyEasy_Very easy,SurveyEasy_Neither easy nor difficult,SurveyEasy_Somewhat difficult,SurveyEasy_nan,SurveyEasy_Very difficult
0,10000.0,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,3.0,5.0,1.0,4.0,6.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,10000.0,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,2.0,3.0,4.0,6.0,1.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,10.0,8.0,5.0,4.0,9.0,1.0,3.0,6.0,2.0,10.0,7.0,1.0,3.0,4.0,10.0,9.0,2.0,6.0,5.0,11.0,7.0,8.0,4.0,2.0,5.0,1.0,3.0,2.0,6.0,7.0,3.0,1.0,5.0,4.0,1.0,3.0,4.0,2.0,7.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,

In [17]:
#欠損値補完の前に欠損値カウント
def null_count(df):
  df["null_count"] = df.isnull().sum(axis=1)
  return df

df_train = null_count(df_train)
df_test = null_count(df_test)
df_train.head()

,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,...,RaceEthnicity_East Asian;Middle Eastern;South Asian,"RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Native American, Pacific Islander, or Indigenous Australian","RaceEthnicity_East Asian;Native American, Pacific Islander, or Indigenous Australian;South Asian","RaceEthnicity_Black or of African descent;East Asian;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",RaceEthnicity_Black or of African descent;East Asian;Hispanic or Latino/Latina;White or of European descent,"RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Native American, Pacific Islander, or Indigenous Australian;White or of European descent",RaceEthnicity_Hispanic or Latino/Latina;South Asian;White or of European descent,RaceEthnicity_Black or of African descent;East Asian;South Asian,"RaceEthnicity_Black or of African descent;East Asian;Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;White or of European descent","RaceEthnicity_Black or of African descent;East Asian;Native American, Pacific Islander, or Indigenous Australian","RaceEthnicity_Native American, Pacific Islander, or Indigenous Australian;South Asian","RaceEthnicity_Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;White or of European descent","RaceEthnicity_Black or of African descent;East Asian;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",RaceEthnicity_Black or of African descent;Hispanic or Latino/Latina;Middle Eastern,"RaceEthnicity_East Asian;Hispanic or Latino/Latina;Middle Eastern;Native American, Pacific Islander, or Indigenous Australian;South Asian;White or of European descent",Age_35 - 44 years old,Age_18 - 24 years old,Age_25 - 34 years old,Age_45 - 54 years old,Age_nan,Age_55 - 64 years old,Age_Under 18 years old,Age_65 years or older,Dependents_Yes,Dependents_No,Dependents_nan,MilitaryUS_nan,MilitaryUS_No,MilitaryUS_Yes,SurveyTooLong_The survey was an appropriate length,SurveyTooLong_The survey was too long,SurveyTooLong_nan,SurveyTooLong_The survey was too short,SurveyEasy_Somewhat easy,SurveyEasy_Very easy,SurveyEasy_Neither easy nor difficult,SurveyEasy_Somewhat difficult,SurveyEasy_nan,SurveyEasy_Very difficult,null_count
0,10000.0,1.0,7.0,10.0,8.0,2.0,5.0,4.0,3.0,6.0,9.0,1.0,5.0,3.0,7.0,10.0,4.0,11.0,9.0,6.0,2.0,8.0,3.0,1.0,5.0,2.0,4.0,1.0,3.0,4.0,5.0,2.0,6.0,7.0,3.0,5.0,1.0,4.0,6.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
1,10000.0,8.0,5.0,7.0,1.0,2.0,6.0,4.0,3.0,10.0,9.0,1.0,10.0,2.0,4.0,8.0,3.0,11.0,7.0,5.0,9.0,6.0,2.0,1.0,4.0,5.0,3.0,7.0,3.0,6.0,2.0,1.0,4.0,5.0,2.0,3.0,4.0,6.0,1.0,7.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0
2,10.0,8.0,5.0,4.0,9.0,1.0,3.0,6.0,2.0,10.0,7.0,1.0,3.0,4.0,10.0,9.0,2.0,6.0,5.0,11.0,7.0,8.0,4.0,2.0,5.0,1.0,3.0,2.0,6.0,7.0,3.0,1.0,5.0,4.0,1.0,3.0,4.0,2.0,7.0,5.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,

In [18]:
#数値型特徴量の欠損値補完を行う。欠損値がある列には欠損フラグの列も追加 #純粋な欠損値だけ見ている。データを確認し、偽装欠損値と見込まれる値が見つかれば別途対応
from sklearn.impute import SimpleImputer

def impute_and_flag(df_train, df_test, col_lis):
  for col in col_lis:
    if df_train[col].isnull().any() == True:
      print("imputed : " + col)
      df_train[col+"_null"] = 0
      df_train.loc[df_train[col].isnull(), col+"_null"] = 1
      df_test[col+"_null"] = 0
      df_test.loc[df_test[col].isnull(), col+"_null"] = 1
      imp = SimpleImputer(missing_values = np.nan, strategy = 'median')
      df_train[col] = imp.fit_transform(df_train[[col]])
      df_test[col] = imp.transform(df_test[[col]])
    elif df_train[col].isnull().any() == True:
      print("missing values is found in only test data for : "+ col)
      #欠損値がTestデータにだけ現れるケースはデータを要確認。そもそもその列は使えないかも。
    else:
      pass
  return df_train, df_test


df_train, df_test = impute_and_flag(df_train, df_test, lis_cols_num)

imputed : AssessJob1
imputed : AssessJob2
imputed : AssessJob3
imputed : AssessJob4
imputed : AssessJob5
imputed : AssessJob6
imputed : AssessJob7
imputed : AssessJob8
imputed : AssessJob9
imputed : AssessJob10
imputed : AssessBenefits1
imputed : AssessBenefits2
imputed : AssessBenefits3
imputed : AssessBenefits4
imputed : AssessBenefits5
imputed : AssessBenefits6
imputed : AssessBenefits7
imputed : AssessBenefits8
imputed : AssessBenefits9
imputed : AssessBenefits10
imputed : AssessBenefits11
imputed : JobContactPriorities1
imputed : JobContactPriorities2
imputed : JobContactPriorities3
imputed : JobContactPriorities4
imputed : JobContactPriorities5
imputed : JobEmailPriorities1
imputed : JobEmailPriorities2
imputed : JobEmailPriorities3
imputed : JobEmailPriorities4
imputed : JobEmailPriorities5
imputed : JobEmailPriorities6
imputed : JobEmailPriorities7
imputed : AdsPriorities1
imputed : AdsPriorities2
imputed : AdsPriorities3
imputed : AdsPriorities4
imputed : AdsPriorities5
impute

In [20]:
#数値型特徴量の標準化　https://scikit-learn.org/stable/modules/classes.html#module-sklearn.preprocessing
from sklearn.preprocessing import StandardScaler

def stand(df_train, df_test, lis_cols_num):
  for col in lis_cols_num:
    scaler = StandardScaler()
    df_train[col] = scaler.fit_transform(df_train[[col]])
    df_test[col] = scaler.transform(df_test[[col]])
  return df_train, df_test

df_train, df_test = stand(df_train, df_test, lis_cols_num)
df_train.head()

AssessJob1
AssessJob2
AssessJob3
AssessJob4
AssessJob5
AssessJob6
AssessJob7
AssessJob8
AssessJob9
AssessJob10
AssessBenefits1
AssessBenefits2
AssessBenefits3
AssessBenefits4
AssessBenefits5
AssessBenefits6
AssessBenefits7
AssessBenefits8
AssessBenefits9
AssessBenefits10
AssessBenefits11
JobContactPriorities1
JobContactPriorities2
JobContactPriorities3
JobContactPriorities4
JobContactPriorities5
JobEmailPriorities1
JobEmailPriorities2
JobEmailPriorities3
JobEmailPriorities4
JobEmailPriorities5
JobEmailPriorities6
JobEmailPriorities7
AdsPriorities1
AdsPriorities2
AdsPriorities3
AdsPriorities4
AdsPriorities5
AdsPriorities6
AdsPriorities7
CompanySize


,CompanySize,AssessJob1,AssessJob2,AssessJob3,AssessJob4,AssessJob5,AssessJob6,AssessJob7,AssessJob8,AssessJob9,AssessJob10,AssessBenefits1,AssessBenefits2,AssessBenefits3,AssessBenefits4,AssessBenefits5,AssessBenefits6,AssessBenefits7,AssessBenefits8,AssessBenefits9,AssessBenefits10,AssessBenefits11,JobContactPriorities1,JobContactPriorities2,JobContactPriorities3,JobContactPriorities4,JobContactPriorities5,JobEmailPriorities1,JobEmailPriorities2,JobEmailPriorities3,JobEmailPriorities4,JobEmailPriorities5,JobEmailPriorities6,JobEmailPriorities7,AdsPriorities1,AdsPriorities2,AdsPriorities3,AdsPriorities4,AdsPriorities5,AdsPriorities6,...,AssessJob2_null,AssessJob3_null,AssessJob4_null,AssessJob5_null,AssessJob6_null,AssessJob7_null,AssessJob8_null,AssessJob9_null,AssessJob10_null,AssessBenefits1_null,AssessBenefits2_null,AssessBenefits3_null,AssessBenefits4_null,AssessBenefits5_null,AssessBenefits6_null,AssessBenefits7_null,AssessBenefits8_null,AssessBenefits9_null,AssessBenefits10_null,AssessBenefits11_null,JobContactPriorities1_null,JobContactPriorities2_null,JobContactPriorities3_null,JobContactPriorities4_null,JobContactPriorities5_null,JobEmailPriorities1_null,JobEmailPriorities2_null,JobEmailPriorities3_null,JobEmailPriorities4_null,JobEmailPriorities5_null,JobEmailPriorities6_null,JobEmailPriorities7_null,AdsPriorities1_null,AdsPriorities2_null,AdsPriorities3_null,AdsPriorities4_null,AdsPriorities5_null,AdsPriorities6_null,AdsPriorities7_null,CompanySize_null
0,2.565072,-2.098154,0.113729,1.570287,1.597608,-0.695998,0.283003,-0.571782,-0.502776,-0.750731,1.132918,-0.404406,-0.315272,-0.470661,-0.245448,1.171932,-0.542144,1.510042,1.224873,-0.805617,-1.987572,0.820193,-0.038269,-0.468916,0.620330,-1.004199,0.377234,-1.643738,-0.792193,0.094038,1.459103,-0.720412,0.641670,1.532712,0.265509,0.699469,-1.508812,0.104873,0.850977,0.986444,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2.565072,0.526787,-0.704897,0.399138,-1.252510,-0.695998,0.699415,-0.571782,-0.502776,0.970816,1.132918,-0.404406,1.389935,-0.842860,-1.352918,0.370758,-0.906396,1.510042,0.468853,-1.184766,0.757096,0.044445,-0.945169,-0.468916,-0.267900,2.117795,-0.668640,1.867140,-0.792193,1.256619,-0.755888,-1.344818,-0.689945,0.048781,-0.318216,-0.505033,0.460340,1.294114,-1.964238,0.986444,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
2,-0.473950,0.526787,-0.704897,-0.772010,2.004768,-1.120035,-0.549822,0.130520,-0.916376,0.970816,0.357241,-0.404406,-0.997355,-0.098462,0.862023,0.771345,-1.270647,-0.398141,-0.287167,1.090131,-0.027095,0.820193,0.868630,0.891555,0.620330,-2.044864,-0.668640,-1.058592,1.180685,1.837909,-0.017558,-1.344818,-0.024138,-0.693185,-0.901941,-0.505033,0.460340,-1.084369,1.414020,-0.189857,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,2.565072,-0.598187,-1.523524,1.179904,-0.031031,-1.120035,1.532240,-1.274084,1.151627,0.970816,-0.030598,-0.404406,-0.997355,-0.842860,0.492866,1.572519,-0.542144,0.365132,0.090843,-0.426467,1.149192,-0.343428,-0.038269,-0.468916,0.620330,1.077130,-1.714513,-1.643738,0.523059,-0.487252,0.720773,-0.720412,0.641670,1.532712,-0.901941,0.097218,-0.852428,0.699493,-0.838152,0.986444,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,-0.446572,-0.223196,-1.523524,0.399138,-0.031031,-1.120035,0.283003,1.535125,1.565228,0.540429,-1.581953,-0.404406,-0.997355,-0.842860,0.492866,1.572519,-0.177892,0.365132,-0.665177,0.710982,-0.027095,0.044445,1.775530,-0.468916,-2.044359,0.036466,0.377234,-0.473446,1.838311,-1.068543,0.720773,-1.344818,0.641670,0.048781,-0.901941,-0.505033,1.116724,0.104873,-1.401195,0.986444,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#パーティショニング (層化抽出)